<a href="https://colab.research.google.com/github/Kshitij0412/ML-projects/blob/main/Copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
train_file_path= "train-data.tsv"
test_file_path = "valid-data.tsv"

In [3]:
df_train = pd.read_table(train_file_path, header=0, names=['type', 'msg'], usecols=['type', 'msg'])
df_test = pd.read_table(test_file_path, header=0, names=['type', 'msg'], usecols=['type', 'msg'])


In [4]:
df_train['type'] = df_train['type'].map({"ham": 0, "spam": 1})
df_test['type'] = df_test['type'].map({"ham": 0, "spam": 1})

In [5]:
tokenizer = tfds.deprecated.text.Tokenizer()
vocabulary_set = set()

In [6]:
for msg in df_train['msg'].values:
    some_tokens = tokenizer.tokenize(msg)
    vocabulary_set.update(some_tokens)

In [7]:
for msg in df_test['msg'].values:
    some_tokens = tokenizer.tokenize(msg)
    vocabulary_set.update(some_tokens)

In [8]:
vocab_size = len(vocabulary_set)
encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set)

In [9]:
def encode(text, label):
    encoded_text = encoder.encode(text.numpy())
    return encoded_text, label

In [10]:
def encode_map_fn(text, label):
    encoded_text, label = tf.py_function(encode, inp=[text, label], Tout=(tf.int64, tf.int64))
    encoded_text.set_shape([None])
    label.set_shape([])
    return encoded_text, label

In [11]:
train_data = tf.data.Dataset.from_tensor_slices((df_train['msg'].values, df_train['type'].values))
test_data = tf.data.Dataset.from_tensor_slices((df_test['msg'].values, df_test['type'].values))

In [12]:
train_data_encoded = train_data.map(encode_map_fn)
test_data_encoded = test_data.map(encode_map_fn)

In [13]:
BUFFER_SIZE = 1000
BATCH_SIZE = 32
train_dataset = train_data_encoded.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE)
test_dataset = test_data_encoded.padded_batch(BATCH_SIZE)

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 32),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [15]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset, validation_steps=30)

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

In [18]:
def predict_message(pred_text):
    pred_text_encoded = encoder.encode(pred_text)
    pred_text_encoded = tf.cast(pred_text_encoded, tf.float32)
    prediction = model.predict(tf.expand_dims(pred_text_encoded, 0)).tolist()
    if prediction[0][0] < 0.5:
        return pred_text, 'ham'
    else:
        return pred_text, 'spam'

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
